# Задание 1

Реализуйте стохастический градиентный спуск для решения линейной регрессии. Исследуйте сходимость с разным размером батча (1 - SGD, 2, $\ldots$, $n - 1$ - Minibatch GD, $n$ - GD из предыдущей работы).

## Реализация

In [59]:
import math
import numpy as np
from tqdm import tqdm

CNST_DPI_HQ = 1024

def exp_decay(epoch, k, initial_lrate=0.3):
   return initial_lrate * math.exp(-0.05 * epoch)

def const_decay(_, __, initial_lrate=0.3):
   return initial_lrate

In [60]:
import random
from helper import *

def stochastic_gradient_descent(f, initial_point, learning_rate_func, learning_rate=0.1, max_epochs=1000, minimum = 0.0, epsilon=0.1, batch_size=1):
    """
    Cтохастический градиентный спуск для поиска минимума функции c learning rate scheduling.

    Аргументы:
        f (function): Изначальная функция.
        grad_fn (function): Функция, которая принимает точку и возвращает градиент в этой точке.
        initial_point (list): Начальную точка, с которой начинается поиск.
        learning_rate_func (function): learning rate scheduling function.
        learning_rate (float): Скорость обучения или шаг градиентного спуска.
        max_epochs (int): Максимальное количество эпох или итераций для выполнения алгоритма.
        minimum (float): Минимум функции.
        epsilon (float): Малое число, используемое как критерий останова для алгоритма.
        batch_size (int): кол-во координат по которым вычисляется градиент
    Возвращает:
        Число пройденных шагов.
    """

    batch_size = min(batch_size, len(initial_point))

    current_point = initial_point.copy()
    current_value = f(current_point)
    steps_count = 0
    
    for epoch in range(max_epochs):
        if abs(current_value - minimum) < epsilon:
            break

        prev_point = np.copy(current_point)

        for _ in range(batch_size):
            random_index = random.randint(0, len(current_point)-1) 
            gradient_random_index = fast_gradient(f, current_point, random_index) 
            current_point[random_index] -= learning_rate_func(epoch, batch_size, learning_rate) * gradient_random_index

        new_value = f(current_point)

        if new_value < current_value:
            current_value = new_value
        else:
            current_point = prev_point

        steps_count += 1
        
    return steps_count

In [61]:
def compare_const_decay_with_exp_decay(f, lr_step_size=1e-1, max_epochs=10000, filename='', filename_extension='.png', dpi=CNST_DPI_HQ):
    lr_values = np.arange(0, 100, lr_step_size)
    x0 = np.array([0, 0], dtype=float)
    batch_size = 2
    learning_rate_functions = [const_decay, exp_decay]
    labels = ['Constant',
              'Exponential']
    

    for i in range(2):
        learning_rate_function = learning_rate_functions[i]
        result = []
    
        for lr_iter in tqdm(lr_values):
            steps = stochastic_gradient_descent(f, x0, learning_rate_function, learning_rate=lr_iter, max_epochs=max_epochs, batch_size=batch_size)
            result.append(steps)

        plt.xlabel('Learning rate', fontsize=14)
        plt.ylabel('Steps', fontsize=14)
        plt.plot(lr_values, result, label=labels[i])

    # min_lr_index = np.argmin(result)
    # min_steps = result[min_lr_index]
    # min_lr = lr_values[min_lr_index]

    # plt.annotate(f'[{min_lr}, {min_steps}]', 
    #             xy=(min_lr, min_steps),
    #             xytext=(min_lr-0.1, min_steps+max_iter/10),
    #             bbox=dict(boxstyle="round", fc="0.8"),
    #             arrowprops=dict(arrowstyle='->', lw=2, color='black'))
    
    # plt.xlabel('Learning rate', fontsize=14)
    # plt.ylabel('Steps', fontsize=14)
    # plt.plot(lr_values, result)
    
    if(filename != ''):
        plt.savefig(filename + filename_extension, dpi=dpi, bbox_inches=0, transparent=True)

    plt.show()

## Тестирование и отладка

In [ ]:
def mse_loss(x):
        weights, bias = x[0], x[1]
        y_pred = np.dot(X, weights) + bias
        mse = np.mean((y - y_pred) ** 2)
        return mse


# ======== style-parameters
plt.style.use('default')
_ = plt.figure(figsize=(8, 8))
# =========================

# Генерируем случайные точки
real_weight, real_bias = 2, 0

dots_count = 500
variance = 3
X = np.random.rand(dots_count, 1)
y = real_weight * X + real_bias + (np.random.rand(dots_count, 1) * variance - variance / 2)
    
compare_const_decay_with_exp_decay(mse_loss)


In [66]:
def mse_loss(x):
        weights, bias = x[0], x[1]
        y_pred = np.dot(X, weights) + bias
        mse = np.mean((y - y_pred) ** 2)
        return mse


# ======== style-parameters
plt.style.use('default')
_ = plt.figure(figsize=(8, 8))
# =========================

# Генерируем случайные точки
real_weight, real_bias = 2, 0

dots_count = 500
variance = 3
X = np.random.rand(dots_count, 1)
y = real_weight * X + real_bias + (np.random.rand(dots_count, 1) * variance - variance / 2)

lr = 45
batch_size = 2
max_epochs = 10_000
x0 = np.array([0, 0], dtype=float)
epsilon = 1e-1
    
const = stochastic_gradient_descent(mse_loss, x0, const_decay, learning_rate=lr, max_epochs=max_epochs, epsilon=epsilon, batch_size=batch_size)
exp = stochastic_gradient_descent(mse_loss, x0, exp_decay, learning_rate=lr, max_epochs=max_epochs, batch_size=batch_size)

print(f'Constant vs Exponential\nLR: {lr}\nConst: {const}\nExponential: {exp}')

Constant vs Exponential
LR: 45
Const: 10000
Exponential: 10000


<Figure size 800x800 with 0 Axes>